In [1]:
#Imports
import numpy as np
import pandas as pd
from tqdm import tnrange, tqdm_notebook


import json
from bs4 import BeautifulSoup, SoupStrainer
import re

import functions 
clean = functions.clean()

In [2]:
att = pd.read_csv("/Users/conorosully/Documents/Legal-Case-Prediction/data/preclean/case_attributes.csv",'#')
ID = list(att['id'])
print(att.columns)
print(len(att))
att.head(1)

Index(['id', 'type', 'url', 'doc-name', 'doc-id-label', 'doc-date', 'vl-type',
       'created-at', 'updated-at', 'app-no-parts', 'app-nos', 'articles',
       'conclusions', 'courts', 'ecli', 'judgement-date', 'judges',
       'last-modified-time', 'nonviolations', 'organisations', 'respondents',
       'separate-opinion', 'violations'],
      dtype='object')
55652


,id,type,url,doc-name,doc-id-label,doc-date,vl-type,created-at,updated-at,app-no-parts,...,courts,ecli,judgement-date,judges,last-modified-time,nonviolations,organisations,respondents,separate-opinion,violations
0,HUDOC-ECHR-1959-001-49198,eu-chrs,http://hudoc.echr.coe.int/app/conversion/docx/...,greece <br>against<br> the united kingdom,176/56,1959-04-20,eu_chr,2017-09-18T10:01:36.112Z,2018-10-26T16:53:58.954Z,"['176', '56']",...,NaN,NaN,NaN,NaN,2018-10-07,NaN,['Committee of Ministers'],['GBR'],NaN,NaN


In [21]:
target = pd.read_csv('/Users/conorosully/Documents/Legal-Case-Prediction/data/clean/target.csv')
print(len(target))
target.head()

8703


,id,date,2,3,5,6,7,8,9,10,...,19,34,35,37,41,46,P1,P4,P12,P7
0,HUDOC-ECHR-1999-001-58225,1999-03-25,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,2,-1,-1,-1,-1,-1
1,HUDOC-ECHR-1999-001-58226,1999-03-25,-1,-1,-1,0,-1,-1,-1,-1,...,-1,-1,-1,-1,2,-1,-1,-1,-1,-1
2,HUDOC-ECHR-1999-001-58227,1999-03-25,-1,-1,-1,2,-1,2,-1,-1,...,-1,-1,2,-1,2,-1,2,-1,-1,-1
3,HUDOC-ECHR-1999-001-58239,1999-04-29,-1,-1,0,-1,-1,-1,-1,-1,...,-1,-1,2,-1,2,-1,-1,-1,-1,-1
4,HUDOC-ECHR-1999-001-58251,1999-05-20,0,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,2,-1,2,-1,-1,-1,-1,-1


In [3]:
titles = []

with open("/Users/conorosully/Documents/Legal-Case-Prediction/data/preclean/titles.txt"
         ) as f:
    for line in f:
        titles.append(line.rstrip('\n'))
len(titles)

256

### Get Judgment Title Variations

In [40]:
#Get variations of Judgements and other documents 
judg_variations = []

for i in tqdm_notebook(ID):
    case = clean.getCase(i)[i]
    
    soup = BeautifulSoup(case)
    titles = soup.find_all(re.compile('^h[1-6]$')) + soup.find_all("p")
    for t in titles:
        result = re.search('>(.*)<', str(t))
        text = result.group(1)
        if ("judgment" in text) and (len(text)<20):
            judg_variations.append(t)


In [11]:
for i in tqdm_notebook(ID):
    case = clean.getCase(i)[i]
    if '<p align="center"><b>judgment of</b></p>' in case:
        clean.renderHTML(i)
        break;

CASE: HUDOC-ECHR-2005-001-69929



case,judgment of,final on,non-pecuniary damages and/or costs,payment on
sapl 37565/97,18/12/2001,18/03/2002,10 500 euros,23/05/2002
desmots andré 41358/98,02/07/2002,06/11/2002,6 000 euros,23/05/2003 + interest
scotti jean‑claude 43719/98,07/01/2003,21/05/2003,500 euros,12/08/2003
kroliczek mieczyslaw 43969/98,02/07/2002,21/05/2003,7 800 euros,07/07/2003
richeux alain 45256/99,12/06/2003,12/09/2003,2 800 euros,27/11/2003
clinique mozart sarl 46098/99,08/06/2004,08/09/2004,9 000 euros,03/12/2004
zuili hector et dominique 46820/99,22/07/2003,22/10/2003,9 54346 euros,19/11/2003
case,judgment of,final on,non-pecuniary damages and/or costs,payment on
faivre jacques 46215/99,17/12/2002,21/05/2003,12 000 euros,01/07/2003
verrerie de biot s.a. 46659/99,27/05/2003,27/08/2003,11 700 euros,01/10/2003


In [8]:
def checkTitle(s):
    s = str(s)
    text = re.search('>(.*)<', s).group(1)

    
    haveTitle = False
    for t in titles:
        if t == s:
            haveTitle =  True
            
    check = ('judgment' in str(s))  and (haveTitle == False) and (len(text)<20)
    
    return check
    
checkTitle('<p align="center">judgment</p>')

False

In [9]:
check_id = []
check_titles = []

for i in tqdm_notebook(ID):
    case = clean.getCase(i)[i]
    soup = BeautifulSoup(case)
    sections = soup.find_all(re.compile('^h[1-6]$')) + soup.find_all("p")
    for s in sections[0:10]:
    
        if checkTitle(s):
            check_titles.append(str(s))
            check_id.append(i)
            
    else:
        continue  # only executed if the inner loop did NOT break
    break
     

### Separate cases into: Judgements and Other 

In [15]:
def checkJudgment(case):
    for t in titles:
        if t in case: return True
    return False
checkJudgment("")

False

In [16]:
judgment_ids = []

for i in tqdm_notebook(ID):
    
    case = clean.getCase(i)[i]
    if checkJudgment(case):
        judgment_ids.append(i)
print(len(judgment_ids))


14108


### Get List of IDs with either a violation or non-violation

In [65]:
v_bool = att["violations"].notnull()
nv_bool = att["nonviolations"].notnull()
judgment_ids_compare = att[v_bool | nv_bool]['id']
len(judgment_ids_compare)

15607

In [57]:
missing = np.setdiff1d(judgment_ids,judgment_ids_compare)
len(missing)

1838

In [62]:
ID = missing[100]
clean.getCase(ID)[ID]
#clean.renderHTML(missing[1])

'<div> <p>information note on the court’s case-law no.</p> <p>november-december 1991</p> <p><i><b>koster v. the netherlands</b></i><b> - 12843/87</b></p> <p>judgment 28.11.1991</p> <p class="section">article 5</p> <p class="section">article 5-3</p> <p class="section">brought promptly before judge or other officer</p> <p>appearance before the military court of a conscript placed in detention on remand during military manoeuvres: <i>violation</i></p> <p>[this summary is extracted from the court’s official reports (series a or reports of judgments and decisions). its formatting and structure may therefore differ from the case-law information note summaries.]</p> <p><b>i.</b><b>article 5 § 3 of the convention</b></p> <p>use in french text of word "<i>aussitôt</i>": confirms that the degree of flexibility in the interpretation and application of the notion of "promptness" is limited – notion to be assessed in each case according to its special features but significance attached to such feat